# NIH awards data
https://exporter.nih.gov/ExPORTER_Catalog.aspx

In [1]:
import requests, zipfile, io
from bs4 import BeautifulSoup
import os

import pandas as pd
import numpy as np

import cleaning_strings as cln

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
url = 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2016.zip'
r = requests.get(url).content
z = zipfile.ZipFile(io.BytesIO(r))
z.extractall()

csv = 'RePORTER_PRJ_C_FY2016.csv'
grants_2016_raw = pd.read_csv(csv, encoding = 'latin1')
os.remove(csv)

pd.set_option('display.max_columns', 50)
grants_2016_raw.head(10)

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,ED_INST_TYPE,FOA_NUMBER,FULL_PROJECT_NUM,FUNDING_ICs,FUNDING_MECHANISM,FY,IC_NAME,NIH_SPENDING_CATS,ORG_CITY,ORG_COUNTRY,ORG_DEPT,ORG_DISTRICT,ORG_DUNS,ORG_FIPS,ORG_NAME,ORG_STATE,ORG_ZIPCODE,PHR,PI_IDS,PI_NAMEs,PROGRAM_OFFICER_NAME,PROJECT_START,PROJECT_END,PROJECT_TERMS,PROJECT_TITLE,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT
0,9115627,K23,GM,4.0,N,7/27/2016,8/1/2016,7/31/2017,859,K23GM104401,SCHOOLS OF MEDICINE,PA-11-009,4K23GM104401-04,NIGMS:194460\,OTHER RESEARCH-RELATED,2016,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,Atherosclerosis; Biotechnology; Cardiovascular...,NEW YORK,UNITED STATES,GENETICS,13.0,78861598,US,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,NY,100296574,PUBLIC HEALTH RELEVANCE: Antiplatelet response...,10799126;,"SCOTT, STUART ALEXANDER;","LONG, ROCHELLE M.",8/1/2013,7/31/2017,ABCB1 gene; Accounting; acute coronary syndrom...,The Pharmacogenomic Control of Clopidogrel Res...,104401.0,GHD,Genetics of Health and Disease Study Section,NaN,NaN,4.0,180500.0,13960.0,194460.0,NaN
1,9128072,R01,NS,4.0,N,8/15/2016,8/1/2016,7/31/2017,853,R01NS085165,SCHOOLS OF MEDICINE,PA-11-260,4R01NS085165-04,NINDS:335781\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF NEUROLOGICAL DISORDERS A...,Biotechnology; Complementary and Alternative M...,BALTIMORE,UNITED STATES,ANESTHESIOLOGY,7.0,188435911,US,UNIVERSITY OF MARYLAND BALTIMORE,MD,212011508,PUBLIC HEALTH RELEVANCE: Activation of microgl...,7017365;,"POLSTER, BRIAN M;","MORRIS, JILL A",9/30/2013,7/31/2018,Acute; analog; Antioxidants; attenuation; Bind...,Novel Mechanisms of Microglial Neurotoxicity a...,85165.0,NOMD,Neural Oxidative Metabolism and Death Study Se...,NaN,NaN,4.0,218750.0,117031.0,335781.0,NaN
2,9056435,D43,TW,4.0,N,3/19/2016,3/1/2016,2/28/2017,989,D43TW009310,SCHOOLS OF MEDICINE,PAR-10-257,4D43TW009310-05,FIC:146822\NIDA:75000\,OTHER RESEARCH-RELATED,2016,FOGARTY INTERNATIONAL CENTER,Aging; Behavioral and Social Science; Brain Di...,ANN ARBOR,UNITED STATES,PSYCHIATRY,12.0,73133571,US,UNIVERSITY OF MICHIGAN,MI,481091276,Although recent science has documented the dev...,7354078; 1862210 (contact);,"WALTON, MAUREEN A; ZUCKER, ROBERT ALPERT (cont...","MICHELS, KATHLEEN M",7/9/2012,2/28/2017,21 year old; Academy; addiction; Address; Alco...,Capacity Building for Lifespan Focused Substan...,9310.0,ZRG1,Special Emphasis Panel,NaN,NaN,5.0,209478.0,12344.0,221822.0,NaN
3,9213716,R01,MH,1.0,N,9/8/2016,9/9/2016,5/31/2017,242,R01MH111502,SCHOOLS OF ARTS AND SCIENCES,RFA-MH-16-775,1R01MH111502-01,NIMH:1593922\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF MENTAL HEALTH,Basic Behavioral and Social Science; Behaviora...,CAMBRIDGE,UNITED STATES,MICROBIOLOGY/IMMUN/VIROLOGY,5.0,82359691,US,HARVARD UNIVERSITY,MA,21385369,Project Narrative To shed light into the basic...,2275890 (contact); 8742217; 6139020;,"DULAC, CATHERINE G (contact); REGEV, AVIV ; ZH...","FREUND, MICHELLE",9/9/2016,5/31/2019,abstracting; Accounting; Address; Aggressive b...,In situ transcriptional analysis of brain circ...,111502.0,ZMH1,Special Emphasis Panel,NaN,NaN,1.0,1077309.0,516613.0,1593922.0,NaN
4,8986215,R01,MH,5.0,N,11/5/2015,12/1/2015,11/30/2016,242,R01MH105329,SCHOOLS OF MEDICINE,PA-14-094,5R01MH105329-02,NIMH:294755\,Non-SBIR/STTR RPGs,2016,NATIONAL INSTITUTE OF MENTAL HEALTH,Acquired Cognitive Impairment; Brain Disorders...,GRAND FORKS,UNITED STATES,OTHER BASIC SCIENCES,0.0,102280781,US,UNIVERSITY OF NORTH DAKOTA,ND,582026059,PUBLIC HEALTH RELEVANCE: The proposed research...,9868481; 2063694 (contact);,"CHEN, XUESONG ; GEIGER, JONATHAN DAVID (contact);","JOSEPH, JEYMOHAN",12/15/2014,11/30/2019,Affect; Anti-Retroviral Agents; antiretroviral...,Role of novel endolysosome-dependent calcium r...,105329.0,ZRG1,Special Emphasis Panel,NaN,NaN,2.0,212500.0,82255.

In [3]:
grants_2016_raw.shape

(71872, 45)

# Processing data

## Basic text processing of column names

In [161]:
grants_2016 = grants_2016_raw.copy()

grants_2016.columns = grants_2016.columns.str.lower()
grants_2016.head(1)

,application_id,activity,administering_ic,application_type,arra_funded,award_notice_date,budget_start,budget_end,cfda_code,core_project_num,ed_inst_type,foa_number,full_project_num,funding_ics,funding_mechanism,fy,ic_name,nih_spending_cats,org_city,org_country,org_dept,org_district,org_duns,org_fips,org_name,org_state,org_zipcode,phr,pi_ids,pi_names,program_officer_name,project_start,project_end,project_terms,project_title,serial_number,study_section,study_section_name,subproject_id,suffix,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,K23,GM,4.0,N,7/27/2016,8/1/2016,7/31/2017,859,K23GM104401,SCHOOLS OF MEDICINE,PA-11-009,4K23GM104401-04,NIGMS:194460\,OTHER RESEARCH-RELATED,2016,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES,Atherosclerosis; Biotechnology; Cardiovascular...,NEW YORK,UNITED STATES,GENETICS,13.0,78861598,US,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,NY,100296574,PUBLIC HEALTH RELEVANCE: Antiplatelet response...,10799126;,"SCOTT, STUART ALEXANDER;","LONG, ROCHELLE M.",8/1/2013,7/31/2017,ABCB1 gene; Accounting; acute coronary syndrom...,The Pharmacogenomic Control of Clopidogrel Res...,104401.0,GHD,Genetics of Health and Disease Study Section,NaN,NaN,4.0,180500.0,13960.0,194460.0,NaN


In [162]:
#converts df to lowercase string
grants_2016 = grants_2016.apply(lambda x: x.astype(str).str.lower())
grants_2016.head(1)

,application_id,activity,administering_ic,application_type,arra_funded,award_notice_date,budget_start,budget_end,cfda_code,core_project_num,ed_inst_type,foa_number,full_project_num,funding_ics,funding_mechanism,fy,ic_name,nih_spending_cats,org_city,org_country,org_dept,org_district,org_duns,org_fips,org_name,org_state,org_zipcode,phr,pi_ids,pi_names,program_officer_name,project_start,project_end,project_terms,project_title,serial_number,study_section,study_section_name,subproject_id,suffix,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,k23,gm,4.0,n,7/27/2016,8/1/2016,7/31/2017,859,k23gm104401,schools of medicine,pa-11-009,4k23gm104401-04,nigms:194460\,other research-related,2016,national institute of general medical sciences,atherosclerosis; biotechnology; cardiovascular...,new york,united states,genetics,13.0,78861598,us,icahn school of medicine at mount sinai,ny,100296574,public health relevance: antiplatelet response...,10799126;,"scott, stuart alexander;","long, rochelle m.",8/1/2013,7/31/2017,abcb1 gene; accounting; acute coronary syndrom...,the pharmacogenomic control of clopidogrel res...,104401.0,ghd,genetics of health and disease study section,nan,nan,4.0,180500.0,13960.0,194460.0,nan


In [163]:
#converts 'nan' to NaN
grants_2016.replace(['nan'], np.nan, inplace=True)
grants_2016.head(1)

,application_id,activity,administering_ic,application_type,arra_funded,award_notice_date,budget_start,budget_end,cfda_code,core_project_num,ed_inst_type,foa_number,full_project_num,funding_ics,funding_mechanism,fy,ic_name,nih_spending_cats,org_city,org_country,org_dept,org_district,org_duns,org_fips,org_name,org_state,org_zipcode,phr,pi_ids,pi_names,program_officer_name,project_start,project_end,project_terms,project_title,serial_number,study_section,study_section_name,subproject_id,suffix,support_year,direct_cost_amt,indirect_cost_amt,total_cost,total_cost_sub_project
0,9115627,k23,gm,4.0,n,7/27/2016,8/1/2016,7/31/2017,859,k23gm104401,schools of medicine,pa-11-009,4k23gm104401-04,nigms:194460\,other research-related,2016,national institute of general medical sciences,atherosclerosis; biotechnology; cardiovascular...,new york,united states,genetics,13.0,78861598,us,icahn school of medicine at mount sinai,ny,100296574,public health relevance: antiplatelet response...,10799126;,"scott, stuart alexander;","long, rochelle m.",8/1/2013,7/31/2017,abcb1 gene; accounting; acute coronary syndrom...,the pharmacogenomic control of clopidogrel res...,104401.0,ghd,genetics of health and disease study section,NaN,NaN,4.0,180500.0,13960.0,194460.0,NaN


In [164]:
#Use this csv to clean PI information
#grants_2016.to_csv('grants_all.csv')

## Selecting columns for analysis
There are a total of 45 columns, which may contain redundant or unnecessary information.

In [165]:
pd.set_option('display.max_colwidth', 5000)
col_info = pd.read_csv('grant_col_info_all.csv')
col_info[:9]

,column_name,descriptions
0,application_id,A unique identifier of the project record in the ExPORTER database.
1,activity,"A 3-character code identifying the grant, contract, or intramural activity through which a project is supported. Within each funding mechanism , NIH uses 3-character activity codes (e.g., F32, K08, P01, R01, T32, etc.) to differentiate the wide variety of research-related programs NIH supports. A comprehensive list of activity codes for grants and cooperative agreements may be found on the Types of Grant Programs Web page. RePORTER also includes R&D contracts (activity codes beginning with the letter N) and intramural projects (beginning with the letter Z)."
2,administering_ic,"Administering Institute or Center - A two-character code to designate the agency,NIH Institute, or Center administering the grant. See Institute/Center code definitions"
3,application_type,"A one-digit code to identify the type of application funded: 1 = New application 2 = Competing continuation (also, competing renewal) 3 = Application for additional (supplemental) support. There are two kinds of type 3competing revisions (which are peer-reviewed and administrative supplements) 4 = Competing extension for an R37 award or first non-competing year of a Fast Track SBIR/STTR award 5 = Non-competing continuation 7 = Change of grantee institution 9 = Change of NIH awarding Institute or Division (on a competing continuation)"
4,arra_funded,“Y” indicates a project supported by funds appropriated through the American Recovery and Reinvestment Act of 2009.
5,award_notice_date,Award notice date or Notice of Grant Award (NGA) is a legally binding document stating the government has obligated funds and which defines the period of support and the terms and conditions of award.\r\n
6,budget_start,The date when a project’s funding for a particular fiscal year begins.
7,budget_end,The date when a project’s funding for a particular fiscal year ends.
8,cfda_code,"Federal programs are assigned a number in the Catalog of Federal Domestic Assistance (CFDA), which is referred to as the ""CFDA code."" The CFDA database helps the Federal government track all programs it has domestically funded. \r\n"


To remove (initial): 5-8, redundant information

In [166]:
to_drop = ['award_notice_date', 'budget_start', 'budget_end', 'cfda_code']
grants_2016 = grants_2016.drop(to_drop, axis = 1)

In [167]:
col_info[9:18]

,column_name,descriptions
9,core_project_num,"An identifier for each research project, used to associate the project with publication and patent records. This identifier is not specific to any particular year of the project. It consists of the project activity code, administering IC, and serial number (a concatenation of Activity, Administering_IC, and Serial_Number). \r\n"
10,ed_inst_type,Generic name for the grouping of components across an institution who has applied for or receives NIH funding. The official name as used by NIH is Major Component Combining Name. \r\n
11,foa_number,"The number of the funding opportunity announcement, if any, under which the project application was solicited. Funding opportunity announcements may be categorized as program announcements, requests for applications, notices of funding availability, solicitations, or other names depending on the agency and type of program. Funding opportunity announcements can be found at Grants.gov/FIND and in the NIH Guide for Grants and Contracts"
12,full_project_num,"Commonly referred to as a grant number, intramural project, or contract number. For grants, this unique identification number is composed of the type code, activity code, Institute/Center code, serial number, support year, and (optional) a suffix code to designate amended applications and supplements."
13,funding_ic(s),"The NIH Institute or Center(s) providing funding for a project are designated by their acronyms (see Institute/Center acronyms ). Each funding IC is followed by a colon (:) and the amount of funding provided for the fiscal year by that IC. Multiple ICs are separated by semicolons (;). Project funding information is available only for NIH, CDC, and FDA projects ."
14,funding_mechanism,"The major mechanism categories used in NIH Budget mechanism tables for the President’s budget. Extramural research awards are divided into three main funding mechanisms: grants, cooperative agreements and contracts. A funding mechanism is the type of funded application or transaction used at the NIH. Within each funding mechanism NIH includes programs. Programs can be further refined by specific activity codes."
15,fy,The fiscal year appropriation from which project funds were obligated.
16,ic_name,"Full name of the administering agency, Institute, or Center."
17,nih_spending_cats,"Congressionally-mandated reporting categories into which NIH projects are categorized. Available for fiscal years 2008 and later. Each project’s spending category designations for each fiscal year are made available the following year as part of the next President’s Budget request. See the Research, Condition, and Disease Categorization System for more information on the categorization process."


To remove: 9 (may add in later if correlating with publications), 10, 11, 16; also 2 (redundant with funding_ics)

In [168]:
to_drop2 = ['administering_ic', 'core_project_num', 'ed_inst_type', 'foa_number', 'ic_name']
grants_2016 = grants_2016.drop(to_drop2, axis = 1)

In [169]:
col_info[18:27]

,column_name,descriptions
18,org_city,"The city in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site. For all NIH intramural projects, Bethesda, MD is used."
19,org_country,The country in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.
20,org_dept,"The departmental affiliation of the contact principal investigator for a project, using a standardized categorization of departments. Names are available only for medical school departments."
21,org_district,The congressional district in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.
22,org_duns,"This field may contain multiple DUNS Numbers separated by a semi-colon. The Data Universal Numbering System is a unique nine-digit number assigned by Dun and Bradstreet Information Services, recognized as the universal standard for identifying and keeping track of business worldwide. \r\n"
23,org_fips,The country code of the grantee organization or contractor as defined in the Federal Information Processing Standard.
24,org_name,"The name of the educational institution, research organization, business, or government agency receiving funding for the grant, contract, cooperative agreement, or intramural project."
25,org_state,The state in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.
26,org_zipcode,The zip code in which the business office of the grantee organization or contractor is located. Note that this may be different from the research performance site.


All redundant: 18, 20-23, 25

In [170]:
to_drop3 = ['org_city', 'org_dept', 'org_district', 'org_duns', 'org_fips', 'org_state']
grants_2016 = grants_2016.drop(to_drop3, axis = 1)

In [171]:
col_info[27:36]

,column_name,descriptions
27,phr,"Submitted as part of a grant application, this statement articulates a project's potential to improve public health."
28,pi_id(s),A unique identifier for each of the project Principal Investigators. Each PI in the RePORTER database has a unique identifier that is constant from project to project and year to year.
29,pi_name(s),The name(s) of the Principal Investigator(s) designated by the organization to direct the research project.
30,program_officer_name,An Institute staff member who coordinates the substantive aspects of a contract from planning the request for proposal to oversight.
31,project_start,"The start date of a project. For subprojects of a multi-project grant, this is the start date of the parent award."
32,project_end,"The current end date of the project, including any future years for which commitments have been made. For subprojects of a multi-project grant, this is the end date of the parent award. Upon competitive renewal of a grant, the project end date is extended by the length of the renewal award."
33,project_terms,"Prior to fiscal year 2008, these were thesaurus terms assigned by NIH CRISP indexers. For projects funded in fiscal year 2008 and later, these are concepts that are mined from the project's title, abstract, and specific aims using an automated text mining tool."
34,project_title,"Title of the funded grant, contract, or intramural (sub)project."
35,serial_number,A six-digit number assigned in serial number order within each administering organization.


27 (phr): maybe later for text analysis, too complex for initial analysis; 29, 34, 35

In [172]:
to_drop4 = ['phr', 'pi_names', 'program_officer_name', 'project_title', 'serial_number']
grants_2016 = grants_2016.drop(to_drop4, axis = 1)

In [173]:
col_info[36:]

,column_name,descriptions
36,study_section,A designator of the legislatively-mandated panel of subject matter experts that reviewed the research grant application for scientific and technical merit.
37,study_section_name,The full name of a regular standing Study Section that reviewed the research grant application for scientific and technical merit. Applications reviewed by panels other than regular standing study sections are designated by “Special Emphasis Panel.”
38,subproject_id,A unique numeric designation assigned to subprojects of a “parent” multi-project research grant.
39,suffix,"A suffix to the grant application number that includes the letter ""A"" and a serial number to identify an amended version of an original application and/or the letter ""S"" and serial number indicating a supplement to the project. ."
40,support_year,"The year of support for a project, as shown in the full project number. For example, a project with number 5R01GM0123456-04 is in its fourth year of support."
41,direct_cost_amt,Total indirect cost funding for a project from all NIH Institute and Centers for a given fiscal year. Costs are available only for NIH awards funded in FY 2012 and onward. Indirect cost amounts are not available for SBIR/STTR awards.
42,indirect_cost_amt,Total indirect cost funding for a project from all NIH Institute and Centers for a given fiscal year. Costs are available only for NIH awards funded in FY 2012 and onward. Indirect cost amounts are not available for SBIR/STTR awards.
43,total_cost,"Total project funding from all NIH Institute and Centers for a given fiscal year. Costs are available only for: NIH, CDC, and FDA grant awards (only the parent record of multi-project grants). -NIH intramural projects (activity codes beginning with “Z”) in FY 2007 and later fiscal years. -NIH contracts (activity codes beginning with “N”) . For multi-project grants, Total_Cost includes funding for all of the constituent subprojects. This field will be blank on subproject records; the total cost of each subproject is found in Total_Cost_Sub_Project ."
44,total_cost_sub_project,Applies to subproject records only. Total funding for a subproject from all NIH Institute and Centers for a given fiscal year. Costs are available only for NIH awards.


37 (redundant with 36, can always look up)
Maybe 38 and 39, depending on how many instances there are

In [174]:
#check number of subprojects
grants_2016.shape
subproject_cols = ['subproject_id', 'suffix', 'total_cost_sub_project']
grants_2016[subproject_cols].isnull().sum()

(71872, 25)

subproject_id             60759
suffix                    61151
total_cost_sub_project    60759
dtype: int64

About 85% of the grants do not have any entries for these three columns; drop for now, along with 37.

In [175]:
grants_2016 = grants_2016.drop(['study_section_name'] + subproject_cols, axis = 1)

In [176]:
pd.set_option('display.max_colwidth', 50)
grants_2016.head()

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_country,org_name,org_zipcode,pi_ids,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,united states,icahn school of medicine at mount sinai,100296574,10799126;,8/1/2013,7/31/2017,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,united states,university of maryland baltimore,212011508,7017365;,9/30/2013,7/31/2018,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,united states,university of michigan,481091276,7354078; 1862210 (contact);,7/9/2012,2/28/2017,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0
3,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,united states,harvard university,21385369,2275890 (contact); 8742217; 6139020;,9/9/2016,5/31/2019,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0
4,8986215,r01,5.0,n,5r01mh105329-02,nimh:294755\,non-sbir/sttr rpgs,2016,acquired cognitive impairment; brain disorders...,united states,university of north dakota,582026059,9868481; 2063694 (contact);,12/15/2014,11/30/2019,affect; anti-retroviral agents; antiretroviral...,zrg1,2.0,212500.0,82255.0,294755.0


Convert strings to numeric and dates

In [177]:
def convert_column(df, list_of_names, function):
    '''
    Converts df column values from x to 'function'.
    Function = numeric: pd.to_numeric()
    Function = date: pd.to_datetime()
    Takes a df and a list with the column names.
    Returns df.
    '''
    for name in list_of_names:
        if function == 'numeric':
            df[name] = pd.to_numeric(df[name])
        elif function == 'date':
            df[name] = pd.to_datetime(df[name], errors = 'coerce')
    return df

numeric_cols = ['direct_cost_amt', 'indirect_cost_amt', 'total_cost']
grants_2016 = convert_column(grants_2016, numeric_cols, function = 'numeric')

date_cols = ['project_start', 'project_end']
grants_2016 = convert_column(grants_2016, date_cols, function = 'date')


grants_2016.head(1)

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_country,org_name,org_zipcode,pi_ids,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,united states,icahn school of medicine at mount sinai,100296574,10799126;,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0


Split grants with multiple PIs so that each row only has a single PI listed.

In [178]:
col_list = ['nih_spending_cats', 'org_name', 'pi_ids', 'project_terms']
grants_2016 = cln.strip_series(grants_2016, col_list, strip = '; ')
grants_2016 = cln.split_rows_pis(grants_2016, col_name = 'pi_ids')

In [179]:
grants_2016 = cln.strip_series(grants_2016, ['pi_ids'])
grants_2016.head()

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_country,org_name,org_zipcode,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_ids
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,united states,icahn school of medicine at mount sinai,100296574,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,10799126
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,united states,university of maryland baltimore,212011508,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0,7017365
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,united states,university of michigan,481091276,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,7354078
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,united states,university of michigan,481091276,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,1862210
3,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,united states,harvard university,21385369,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0,2275890


Split grant totals by number of associated PIs (assumption is that all PIs on a grant receive the same amount of money).

In [180]:
#divide cost columns by # of times the application_id occurs
pi_per_grant = pd.DataFrame(grants_2016['application_id'].value_counts())
pi_per_grant = pi_per_grant.reset_index()
pi_per_grant.columns = ['application_id', 'num_pis']

#Match application IDs in grants_2016 and pi_per_grant
#Divide the 3 cost columns by 
grants_2016 = pd.merge(grants_2016, pi_per_grant, on = 'application_id')

In [181]:
#grants_2016.rename(columns = {'num_pis_x':'num_pis'}, inplace = True) 
#del grants_2016['num_pis_y']
grants_2016.head()

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_country,org_name,org_zipcode,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_ids,num_pis
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,united states,icahn school of medicine at mount sinai,100296574,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,10799126,1
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,united states,university of maryland baltimore,212011508,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0,7017365,1
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,united states,university of michigan,481091276,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,7354078,2
3,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,united states,university of michigan,481091276,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,209478.0,12344.0,221822.0,1862210,2
4,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,united states,harvard university,21385369,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,1077309.0,516613.0,1593922.0,2275890,3


In [182]:
def divide_by_column(df, col_list = ['direct_cost_amt', 'indirect_cost_amt', 'total_cost'], divide_by = 'num_pis'):
    '''
    
    Divide df columns by value in another column
    
    '''
    for col in col_list:
        df[col] = round(df[col] / df[divide_by])
    return df

grants_2016 = divide_by_column(grants_2016)

In [183]:
grants_2016.head()

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_country,org_name,org_zipcode,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_ids,num_pis
0,9115627,k23,4.0,n,4k23gm104401-04,nigms:194460\,other research-related,2016,atherosclerosis; biotechnology; cardiovascular...,united states,icahn school of medicine at mount sinai,100296574,2013-08-01,2017-07-31,abcb1 gene; accounting; acute coronary syndrom...,ghd,4.0,180500.0,13960.0,194460.0,10799126,1
1,9128072,r01,4.0,n,4r01ns085165-04,ninds:335781\,non-sbir/sttr rpgs,2016,biotechnology; complementary and alternative m...,united states,university of maryland baltimore,212011508,2013-09-30,2018-07-31,acute; analog; antioxidants; attenuation; bind...,nomd,4.0,218750.0,117031.0,335781.0,7017365,1
2,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,united states,university of michigan,481091276,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,104739.0,6172.0,110911.0,7354078,2
3,9056435,d43,4.0,n,4d43tw009310-05,fic:146822\nida:75000\,other research-related,2016,aging; behavioral and social science; brain di...,united states,university of michigan,481091276,2012-07-09,2017-02-28,21 year old; academy; addiction; address; alco...,zrg1,5.0,104739.0,6172.0,110911.0,1862210,2
4,9213716,r01,1.0,n,1r01mh111502-01,nimh:1593922\,non-sbir/sttr rpgs,2016,basic behavioral and social science; behaviora...,united states,harvard university,21385369,2016-09-09,2019-05-31,abstracting; accounting; address; aggressive b...,zmh1,1.0,359103.0,172204.0,531307.0,2275890,3


Checking non-US countries, we see there are many NaNs.

In [219]:
grants_2016.ix[grants_2016['org_country'] != 'united states']

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_country,org_name,org_zipcode,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_ids,num_pis
19,9359909,zia,1.0,n,1ziatr000230-02,ncats:226982\,intramural research,2016,biotechnology; cancer,NaN,translational science,NaN,NaT,NaT,analog; antineoplastic agents; assay developme...,NaN,2.0,NaN,NaN,226982.0,14380202,1
20,9359855,zia,1.0,n,1zialm010001-07,nlm:530704\,intramural research,2016,networking and information technology r&d,NaN,national library of medicine,NaN,NaT,NaT,abstracting; academia; anatomy; base; bioimagi...,NaN,7.0,NaN,NaN,530704.0,10687426,1
21,9351937,zia,1.0,n,1ziaag000262-09,nia:202174\,intramural research,2016,aging; assistive technology; bioengineering; c...,NaN,aging,NaN,NaT,NaT,adult; age; age difference; age related; aged;...,NaN,9.0,NaN,NaN,202174.0,9691085,1
27,9352075,zia,1.0,n,1ziadc000064-16,nidcd:891718\,intramural research,2016,clinical research; congenital structural anoma...,NaN,deafness & other communication disorders,NaN,NaT,NaT,acoustic nerve; action potentials; adolescent;...,NaN,16.0,NaN,NaN,891718.0,10271451,1
45,9354094,zia,1.0,n,1ziadc000009-12,nidcd:473676\,intramural research,2016,neurosciences,NaN,deafness & other communication disorders,NaN,NaT,NaT,accounting; actins; adhesives; alloys; alpha a...,NaN,12.0,NaN,NaN,473676.0,6569943,1
86,9343907,zia,1.0,n,1ziabc011407-06,nci:13322\,intramural research,2016,brain cancer; brain disorders; cancer; clinica...,NaN,basic sciences,NaN,NaT,NaT,bioinformatics; biology; biopsy; brain stem gl...,NaN,6.0,NaN,NaN,13322.0,9692423,1
92,9197580,r01,1.0,n,1r01hd089685-01,nichd:341190\,non-sbir/sttr rpgs,2016,clinical research; contraception/reproduction;...,australia,university of adelaide,5005,2016-09-23,2020-05-31,affect; african american; australia; biobank; ...,zhd1,1.0,315917.0,25273.0,341190.0,12394158,1
98,9357197,zia,1.0,n,1ziacl003007-03,NaN,intramural research,2016,NaN,NaN,clinical center,NaN,NaT,NaT,affect; animals; apoptosis; apoptotic; area; b...,NaN,3.0,NaN,NaN,NaN,6480161,1
104,9363639,zia,1.0,n,1ziaaa000125-01,niaaa:2404834\,intramural research,2016,"alcoholism, alcohol use and health; basic beha...",NaN,alcohol abuse and alcoholism,NaN,NaT,NaT,abstinence; addiction; alcohol abuse; alcohol ...,NaN,1.0,NaN,NaN,2404834.0,14282131,1
125,9056839,u2g,5.0,n,5u2ggh000105-05,cogh:4939786\,others,2016,NaN,zambia,eastern provincial health office,10101,2011-09-30,2017-03-31,NaN,zgh1,5.0,NaN,NaN,4939786.0,12250037,1


In [149]:
grants_2016.ix[grants_2016['org_country'] != 'united states'].org_country.value_counts()

south africa      170
canada            157
united kingdom     88
uganda             54
botswana           37
ethiopia           36
switzerland        35
kenya              31
nigeria            31
brazil             28
germany            28
peru               26
australia          26
india              24
zambia             22
france             20
tanzania u rep     20
thailand           20
china              19
mozambique         19
ghana              15
zimbabwe           12
mali               12
argentina          11
cote d'ivoire      11
netherlands        10
haiti              10
malawi             10
israel              9
colombia            9
                 ... 
costa rica          2
congo dem rep       2
sri lanka           2
austria             2
senegal             2
tajikistan          2
georgia             2
iceland             2
nicaragua           2
jordan              2
hong kong           2
japan               2
panama              2
kazakhstan          1
fed micron

In [190]:
#are all null countries from NIH?
null_country = grants_2016.ix[grants_2016['org_country'].isnull()]
null_country['org_name'].value_counts(dropna = False)

basic sciences                                                                   676
niaid extramural activities                                                      363
diabetes, digestive, kidney diseases                                             267
aging                                                                            201
heart, lung, and blood institute                                                 195
translational science                                                            175
child health and human development                                               154
clinical center                                                                  139
environmental health sciences                                                    139
neurological disorders and stroke                                                108
national eye institute                                                           101
national institute of mental health                              

All the listed organization names are associated with the NIH, including omnitec solutions. Replace organization information with that of the NIH in pi_info (in notebook cleaning_pi_info). Will eventually replace the organization information in grants_2016 with that from pi_info.csv.

In [193]:
null_country.ix[null_country['org_name'].isnull()]

,application_id,activity,application_type,arra_funded,full_project_num,funding_ics,funding_mechanism,fy,nih_spending_cats,org_country,org_name,org_zipcode,project_start,project_end,project_terms,study_section,support_year,direct_cost_amt,indirect_cost_amt,total_cost,pi_ids,num_pis
958,9203594,n01,NaN,n,272201400027c-3-0-1,niaid:5800429\,non sbir/sttr contracts,2016,antimicrobial resistance; biodefense; emerging...,NaN,NaN,NaN,2015-09-15,2016-09-14,antimicrobial resistance; archives; basic scie...,NaN,NaN,NaN,NaN,5800429.0,12426198,1
1756,9369771,n02,NaN,n,263201100104u-15-0-1,nei:99253\,non sbir/sttr contracts,2016,eye disease and disorders of vision; neuroscie...,NaN,NaN,NaN,2011-02-07,2016-02-06,contracts; disease model; drug development; ed...,NaN,NaN,NaN,NaN,99253.0,14368787,1
5135,9320261,n02,NaN,n,261201300002c-8-0-1,od:50000\,non sbir/sttr contracts,2016,biodefense; cancer; clinical research; conditi...,NaN,NaN,NaN,2013-02-15,2017-02-14,accidents; affect; age; area; belarus; cohort;...,NaN,NaN,NaN,NaN,50000.0,12377190,1
6923,9326094,n01,NaN,n,271201300028c-8-0-1,nimh:622593\,non sbir/sttr contracts,2016,brain disorders; cerebrovascular; clinical res...,NaN,NaN,NaN,2016-09-01,2017-08-31,advocacy organizations; award; base; biologica...,NaN,NaN,NaN,NaN,622593.0,12627838,1
11632,9367518,n02,NaN,n,273201600003p-0-0-1,niehs:125000\,non sbir/sttr contracts,2016,NaN,NaN,NaN,NaN,2015-10-16,2016-10-15,biological; biological assay; biological avail...,NaN,NaN,NaN,NaN,125000.0,14767866,1
21069,9272292,n01,NaN,n,272201500014c-2-0-1,niaid:1634580\,non sbir/sttr contracts,2016,antimicrobial resistance; biodefense; emerging...,NaN,NaN,NaN,2015-07-01,2017-12-12,acinetobacter baumannii; bacterial resistance;...,NaN,NaN,NaN,NaN,1634580.0,14164955,1
25342,9238586,n02,NaN,n,271201300224p-5-0-1,ninds:18003\,non sbir/sttr contracts,2016,neurosciences,NaN,NaN,NaN,2016-02-28,2017-02-27,address; biological assay; biology; contracts;...,NaN,NaN,NaN,NaN,18003.0,12674785,1
35105,9233834,n02,NaN,n,271201300222p-7-0-1,ninds:18000\,non sbir/sttr contracts,2016,neurosciences,NaN,NaN,NaN,2016-02-28,2017-02-27,address; biological assay; biology; contracts;...,NaN,NaN,NaN,NaN,18000.0,12674789,1
37463,9355364,n01,NaN,n,nihsassfreserv6-0-0-1,nia:27351\,non sbir/sttr contracts,2016,aging,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,27351.0,,1
41215,9360222,n02,NaN,n,261201600003c-0-0-1,nci:1160881\,non sbir/sttr contracts,2016,cancer; patient safety,NaN,NaN,NaN,2016-06-16,2017-05-31,abstracting; cancer center; cancer therapy eva...,NaN,NaN,NaN,NaN,1160881.0,14600584,1


All grants with no listed country and no listed organization are of the sbir/sttr contract form. There are only 25 total with no information, however, they account for a non-trivial amount of money, so will keep these 25 observations.

Match pi_ids in grants_2016 with pi_ids in pi_info.csv and replace org_country, org_name and org_zipcode with information in pi_info.csv.

In [220]:
test = grants_2016.copy()
pi_info = pd.read_csv('pi_info.csv', compression = 'gzip')
pi_info.head()

,pi_ids,pi_names,org_name,org_city,org_state,org_country,org_zipcode
0,10799126.0,"scott, stuart alexander",icahn school of medicine at mount sinai,NaN,NaN,NaN,NaN
1,7017365.0,"polster, brian m",university of maryland baltimore,NaN,NaN,NaN,NaN
2,1866930.0,"so, peter t",massachusetts institute of technology,NaN,NaN,NaN,NaN
3,11044822.0,"gade, terence p",university of pennsylvania,NaN,NaN,NaN,NaN
4,9335858.0,"qi, ling",cornell university,NaN,NaN,NaN,NaN


Save dataframe to csv

In [67]:
grants_2016.to_csv('for_analysis.csv', index = False, compression = 'gzip')

# Initial data exploration

In [654]:
#count all NaNs
grants_2016.isnull().sum()

activity                   17
application_type         1627
arra_funded                17
full_project_num            0
funding_ics               155
funding_mechanism          33
fy                          0
nih_spending_cats        9490
org_city                 3345
org_name                  106
org_state                4132
pi_names                    0
program_officer_name    18412
project_start           13212
project_end             12982
project_terms            2803
study_section            5499
support_year             1627
direct_cost_amt         10674
indirect_cost_amt       15920
total_cost              13146
dtype: int64

In [ ]:
#geospatial data
#Questions for analysis: how many grants are solo vs. joint? how many PIs have solo vs. joint grants?